Name & Student Number:
▪ Ryan Alexander – 1827474
▪ Xin Nian – 1760829
▪ Kineta Padiachee – 1853416
▪ Bianco Van Der Ross – 1827456

# Multinomial Naives Bayes 

In [472]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator
import re

In [473]:
#imports the dataset file using panda 
df = pd.read_csv('StackOverflow-Dataset-cut.csv',delimiter = ';')
print(df.shape)#displayes how many elements are present withhin the file and how many classes there are
df.tail()

(1033, 2)


,title,label
1028,Method launched by another class doesn&#39;t work,python
1029,How to plot integrating over a slice of a two ...,python
1030,how can I print a value that correspond to a d...,python
1031,Put legend for all lines on the matplolit,python
1032,Delayed time series prediction,python


In [474]:
df['label'].value_counts(normalize=True)#displays the ratio of classes present within thhe dataset 

python        0.504356
javascript    0.495644
Name: label, dtype: float64

In [475]:
# Randomize the dataset
data_randomized = df.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.75)#75%training data with 25 percent testing data

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)#displayes how many elements are present withhin the training data and how many classes there are
print(test_set.shape)#displayes how many elements are present withhin the test data and how many classes there are

(775, 2)
(258, 2)


In [476]:
training_set['label'].value_counts(normalize=True)
#displays the ratio of classes present within the training set 

python        0.505806
javascript    0.494194
Name: label, dtype: float64

In [477]:
test_set['label'].value_counts(normalize=True)
#displays the ratio of classes present within the test se

javascript    0.5
python        0.5
Name: label, dtype: float64

In [478]:
training_set.tail(3)#displays tail/end of training set before removing punction and lower casing

,title,label
772,How can a make new list from a 2d list with sp...,python
773,Python3 commmands not working in my virtual en...,python
774,Iterate over DOM elements and dynamically add ...,javascript


In [479]:
training_set['title'] = training_set['title'].str.replace(   '\W', ' ') # Removes punctuation
training_set['title'] = training_set['title'].str.lower()#sets all text to lower case
training_set.tail(3)#displays tail/end of training set after removing punction and lower casing

,title,label
772,how can a make new list from a 2d list with sp...,python
773,python3 commmands not working in my virtual en...,python
774,iterate over dom elements and dynamically add ...,javascript


In [480]:
training_set['title'] = training_set['title'].str.split()

vocabulary = []#creates a empty list to be used to make an vuild a vocabulary/dictionary 
for title in training_set['title']:
    for word in title:
        vocabulary.append(word)#adds each word present in the training dataset to the vocabulary list 

vocabulary = list(set(vocabulary))

In [481]:
len(vocabulary)#prints out length of vocabulary


2036

In [482]:
word_counts_per_title = {unique_word: [0] * len(training_set['title']) for unique_word in vocabulary}

for index, title in enumerate(training_set['title']):
    for word in title:
        word_counts_per_title[word][index] += 1

In [483]:
word_counts = pd.DataFrame(word_counts_per_title)


In [484]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,title,label,remove,pagination,last,parse,system,sklearn,known,automation,...,formset,rotating,divs,divided,alphabetical,animation,parent,color,bashoperator,decrease
0,"[python, how, do, i, output, all, the, text, c...",python,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[scrollby, invoked, through, execute_script, i...",python,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[conditionnally, change, color, on, multi, sel...",javascript,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[vuejs, sever, side, rendering, computed, prop...",javascript,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[finding, out, if, a, user, has, a, certain, r...",javascript,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [485]:
# Isolating python and java title first
python_title = training_set_clean[training_set_clean['label'] == 'python']
java_title = training_set_clean[training_set_clean['label'] == 'javascript']

# P(python) and P(java) /Probability of having one oe the other label
p_python = len(python_title) / len(training_set_clean)
p_java = len(java_title) / len(training_set_clean)

# N_python
n_words_per_python_title = python_title['title'].apply(len)
n_python = n_words_per_python_title.sum()

# N_java
n_words_per_java_title = java_title['title'].apply(len)
n_java = n_words_per_java_title.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [486]:
# Initiate parameters
parameters_python = {unique_word:0 for unique_word in vocabulary}
parameters_java = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_python = python_title[word].sum() # python_title already defined
    p_word_given_python = (n_word_given_python + alpha) / (n_python + alpha*n_vocabulary)
    parameters_python[word] = p_word_given_python

    n_word_given_java = java_title[word].sum() # java_title already defined
    p_word_given_java = (n_word_given_java + alpha) / (n_java + alpha*n_vocabulary)
    parameters_java[word] = p_word_given_java

In [487]:
def classify_test_set(title):
    title = re.sub('\W', ' ', title)#removes punctioation from test data 
    title = title.lower().split()#lower cases the test data

    p_python_given_title = p_python
    p_java_given_title = p_java

    for word in title:
        if word in parameters_python:
            p_python_given_title *= parameters_python[word]#checks probability of title in python vocabulary

        if word in parameters_java:
            p_java_given_title *= parameters_java[word]#checks probability of title in python vocabulary

    if p_java_given_title > p_python_given_title:#if probability of the title is higher in the javascript vocabulary then assign predicted to javascript 
        return 'javascript'
    elif p_python_given_title > p_java_given_title:#if probability of the title is higher in the python vocabulary then assign predicted to python
        return 'python'
    else:
        return 'needs human classification'#if probability of the title is equal to eachothher then then error occurs and requires human intervention

In [488]:
test_set['predicted'] = test_set['title'].apply(classify_test_set)#adds and prints all the predicted outcomes to its actual outcomes
test_set.head()

,title,label,predicted
0,Add multiple values in a row of a dataframe wh...,python,python
1,Passing jQuery data table row as argument to J...,javascript,javascript
2,Recursively build a query string from JSON object,javascript,javascript
3,onclick=&quot;doSomething([object Object])&quo...,javascript,python
4,Is it possible to un-babelify (un-transform) a...,javascript,javascript


In [489]:
y_actual = []
y_pred = []
for row in test_set.iterrows():
    row = row[1]
    y_actual= test_set['label'].apply(lambda x:0 if x=='python' else 1)
    y_pred=test_set['predicted'].apply(lambda x:0 if x=='python' else 1)
    
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['label'] == row['predicted']:
      correct += 1
    
    #creates confusion matrix and prints the accuracy 
print("Confusion Matrix:")
y_actual = pd.Series(y_actual, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
df_confusion = pd.crosstab(y_actual,y_pred)
print(df_confusion)

print("-------------------------")
print('Accuracy:', correct/total)

Confusion Matrix:
Predicted    0    1
Actual             
0          104   25
1           13  116
-------------------------
Accuracy: 0.8527131782945736
